In [20]:
require 'nn'
require 'hdf5'
require 'optim'

In [29]:
myFile = hdf5.open('6-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [30]:
data

{
  train_nocounts : DoubleTensor - size: 887522x6
  train_1000 : DoubleTensor - size: 887522x6
  test : LongTensor - size: 3761x55
  train : LongTensor - size: 772670x7
  nwords : LongTensor - size: 1
  valid : LongTensor - size: 3370x55
  train_1000_nocounts : LongTensor - size: 696825x7
  valid_output : LongTensor - size: 3370x50
}


In [23]:
nwords = 10001
train = data['train_nocounts']:narrow(2,1,5)
train_input = train:narrow(2,1,4)
train_output = train:narrow(2,5,1)

valid_topredict = data['valid']:narrow(2,1,50)
valid_input = data['valid']:narrow(2,51,4)
valid_output = data['valid_output']

In [5]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train_input[i]:view(1,4), train_output[i]}
end
function dataset:size() return train:size(1) end

In [54]:
N = 5
dwin = N-1
hid1 = 25.
hid2 = 80
dnnlm = nn.Sequential()

LT = nn.Sequential()
LT_ = nn.LookupTable(nwords,hid1)
LT:add(LT_)
LT:add(nn.View(1,-1,hid1*dwin))
LT:add(nn.Squeeze()) 

dnnlm:add(LT)

concat = nn.ConcatTable()

lin_tanh = nn.Sequential()
lin_tanh:add(nn.Linear(hid1*dwin,hid2))
lin_tanh:add(nn.Tanh())

id = nn.Identity()

concat:add(lin_tanh)
concat:add(id)

dnnlm:add(concat)
dnnlm:add(nn.JoinTable(1))
dnnlm:add(nn.Linear(hid2+hid1*dwin, nwords))
dnnlm:add(nn.LogSoftMax())

In [55]:
criterion = nn.ClassNLLCriterion()

In [51]:
kag_pred_valid = torch.Tensor(valid_input:size(1),50)
norm_mat = torch.zeros(valid_input:size(1),50)

In [57]:
for i = 1, 1 do
    
    timer = torch.Timer()
    
    LT_.weight:renorm(2,1,1)
    
    trainer = nn.StochasticGradient(dnnlm, criterion)
    trainer.learningRate = 0.005
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
    -- Evaluatin perplexity on validation:
    kag_pred_valid:zero()

    for i = 1, valid_input:size(1) do
        kag_pred_valid[i]:copy(dnnlm:forward(valid_input[i]):index(1, valid_topredict[i])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for i = 1, valid_input:size(1) do
        mm,aa = valid_output[i]:max(1)
        CE = CE + math.log(kag_pred_valid[i][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

# StochasticGradient: training	


# current error = 6.0318624286766	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.0318624286766	
Epoch 1: 3777.7758259773	


Perplexity on valid: 5.8055104706363	


In [58]:
for i = 2, 9 do
    
    timer = torch.Timer()
    
    LT_.weight:renorm(2,1,1)
    
    trainer = nn.StochasticGradient(dnnlm, criterion)
    trainer.learningRate = 0.005
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
    -- Evaluatin perplexity on validation:
    kag_pred_valid:zero()

    for i = 1, valid_input:size(1) do
        kag_pred_valid[i]:copy(dnnlm:forward(valid_input[i]):index(1, valid_topredict[i])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for i = 1, valid_input:size(1) do
        mm,aa = valid_output[i]:max(1)
        CE = CE + math.log(kag_pred_valid[i][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

# StochasticGradient: training	


# current error = 5.5262946470065	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.5262946470065	
Epoch 2: 3650.7690629959	


Perplexity on valid: 5.367000690273	


# StochasticGradient: training	


# current error = 5.3135841631288	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.3135841631288	
Epoch 3: 3620.9383718967	


...

# current error = 4.8692454295903	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.8692454295903	
Epoch 8: 3622.6675741673	


Perplexity on valid: 5.0872367119564	


# StochasticGradient: training	


# current error = 4.8223222568596	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.8223222568596	
Epoch 9: 3623.1487159729	


Perplexity on valid: 5.0862021600192	


In [59]:
for i = 1,1  do
    
    timer = torch.Timer()
    
    LT_.weight:renorm(2,1,1)
    
    trainer = nn.StochasticGradient(dnnlm, criterion)
    trainer.learningRate = 0.001
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
    -- Evaluatin perplexity on validation:
    kag_pred_valid:zero()

    for i = 1, valid_input:size(1) do
        kag_pred_valid[i]:copy(dnnlm:forward(valid_input[i]):index(1, valid_topredict[i])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for i = 1, valid_input:size(1) do
        mm,aa = valid_output[i]:max(1)
        CE = CE + math.log(kag_pred_valid[i][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

# StochasticGradient: training	


# current error = 4.7050212120435	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.7050212120435	
Epoch 1: 3821.8381781578	


Perplexity on valid: 4.9898126480392	


In [ ]:
N = 5
dwin = N-1
hid1 = 25.
hid2 = 80
nce = nn.Sequential()

LTnce = nn.Sequential()
LT_nce = nn.LookupTable(nwords,hid1)
LTnce:add(LT_nce)
LTnce:add(nn.View(1,-1,hid1*dwin))
LTnce:add(nn.Squeeze()) 

nce:add(LTnce)

concatnce = nn.ConcatTable()

lin_tanhnce = nn.Sequential()
lin_tanhnce:add(nn.Linear(hid1*dwin,hid2))
lin_tanhnce:add(nn.Tanh())

idnce = nn.Identity()

concatnce:add(lin_tanhnce)
concatnce:add(id)

nce:add(concatnce)
nce:add(nn.JoinTable(1))
final_linear = nn.Linear(hid2+hid1*dwin, nwords)
nce:add(final_linear)
nce:add(nn.LogSoftMax())

### By Hand:

In [31]:
nwords = 10001
train = data['train']:narrow(2,1,6)
train_input = train:narrow(2,1,5)
train_output = train:narrow(2,6,1)

valid_topredict = data['valid']:narrow(2,1,50)
valid_input = data['valid']:narrow(2,51,5)
valid_output = data['valid_output']

In [32]:
batchSize = 32
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100
nnlm = nn.Sequential()

LT2 = nn.Sequential()
LT_2 = nn.LookupTable(nwords,hid1)
LT2:add(LT_2)
LT2:add(nn.View(1,-1,hid1*dwin))
LT2:add(nn.Squeeze()) 

nnlm:add(LT2)

concat2 = nn.ConcatTable()

lin_tanh2 = nn.Sequential()
lin_tanh2:add(nn.Linear(hid1*dwin,hid2))
lin_tanh2:add(nn.Tanh())

id2 = nn.Identity()

concat2:add(lin_tanh2)
concat2:add(id2)

nnlm:add(lin_tanh2)
-- nnlm:add(nn.JoinTable(2))
nnlm:add(nn.Linear(hid2, nwords))
nnlm:add(nn.LogSoftMax())

parameters,gradParameters = nnlm:getParameters()

criterion = nn.ClassNLLCriterion()

In [ ]:
kag_pred_valid = torch.Tensor(valid_input:size(1),50)
norm_mat = torch.zeros(valid_input:size(1),50)

for i = 1,5  do
    
    timer = torch.Timer()
    
    LT_2.weight:renorm(2,1,1)
    
    av_L = 0
    for t = 1, train_input:size(1), batchSize do
      -- create mini batch
        inputs = torch.Tensor(batchSize,dwin)
        targets = torch.Tensor(batchSize)
        k = 1
        for i = t,math.min(t+batchSize-1,train_input:size(1)) do
         -- load new sample
            inputs[k] = train_input[i]
            targets[k] = train_output[i]
            k = k + 1
        end
        
        gradParameters:zero()
        outputs = nnlm:forward(inputs:narrow(1,1,k-1))
        L = criterion:forward(outputs:narrow(1,1,k-1), targets:narrow(1,1,k-1))
        dL = criterion:backward(outputs:narrow(1,1,k-1), targets:narrow(1,1,k-1))
        nnlm:backward(inputs:narrow(1,1,k-1), dL)
        
        nnlm:updateParameters(0.1)
        
        av_L = av_L + L
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Loss: '..av_L/math.floor(train_input:size(1)/batchSize))
    
    -- Evaluatin perplexity on validation:
    kag_pred_valid:zero()

    for i = 1, valid_input:size(1) do
        kag_pred_valid[i]:copy(nnlm:forward(valid_input[i]):index(1, valid_topredict[i])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for i = 1, valid_input:size(1) do
        mm,aa = valid_output[i]:max(1)
        CE = CE + math.log(kag_pred_valid[i][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

Epoch 1: 432.65068387985	
Average Loss: 6.3998842631204	


Perplexity on valid: 7.231744842634	


Epoch 2: 466.20021295547	
Average Loss: 5.931770989839	


Perplexity on valid: 6.711158500633	


Perplexity on valid: 4.9898126480392	


### Comparision with stochgrad:

In [101]:
batchSize = 32
N = 5
dwin = N-1
hid1 = 25.
hid2 = 80
nnlm = nn.Sequential()

LT2 = nn.Sequential()
LT_2 = nn.LookupTable(nwords,hid1)
LT2:add(LT_2)
LT2:add(nn.View(1,-1,hid1*dwin))
LT2:add(nn.Squeeze()) 

nnlm:add(LT2)

concat2 = nn.ConcatTable()

lin_tanh2 = nn.Sequential()
lin_tanh2:add(nn.Linear(hid1*dwin,hid2))
lin_tanh2:add(nn.Tanh())

id2 = nn.Identity()

concat2:add(lin_tanh2)
concat2:add(id2)

nnlm:add(concat2)
nnlm:add(nn.JoinTable(1))
nnlm:add(nn.Linear(hid2+hid1*dwin, nwords))
nnlm:add(nn.LogSoftMax())

parameters,gradParameters = nnlm:getParameters()

In [98]:
dataset2={};
for i=1,train:size(1) do 
  dataset2[i] = {train_input[i]:view(1,4), train_output[i]}
end
function dataset2:size() return train:size(1) end

In [102]:
for i = 1, 1 do
    
    timer = torch.Timer()
    
    LT_2.weight:renorm(2,1,1)
    
    trainer = nn.StochasticGradient(nnlm, criterion)
    trainer.learningRate = 0.005
    trainer.maxIteration = 1
    trainer:train(dataset2)
    print('Epoch '..i..': '..timer:time().real)
    
end

# StochasticGradient: training	


# current error = 4.450221031528	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.450221031528	
Epoch 1: 474.23745799065	
